In [1]:
from typing import Tuple
import numpy as np
import deepgp

from GPy.models import GPRegression
from emukit.test_functions import forrester_function
from emukit.core.initial_designs import RandomDesign
from emukit.model_wrappers import GPyModelWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement, NegativeLowerConfidenceBound, ProbabilityOfImprovement
from emukit.core.optimization import GradientAcquisitionOptimizer
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity
from gpflow.kernels import RBF, White, Linear
from tqdm import tqdm

In [ ]:
from simulator import MainSimulator, TinySimulator
from world import DebugInfo
from pprint import pprint

main_simulator = MainSimulator()

mutation_rates = {
    "size": 1,
    "speed": 0,
    "vision": 0,
    "aggression": 0
}

days_log = []
for i in tqdm(range(50)):
    main_simulator = MainSimulator()
    days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
        period=10, should_display_day=True, should_display_grid=False, should_display_traits=False), max_days=1000)
    days_log.append(days_survived)
    print(days_survived)


# for log_item in log[-3:-1]:
#     print(log_item)

  0%|                                                                                                                                                      | 0/50 [00:00<?, ?it/s]

Day number: 10
Day number: 20
Day number: 30
Day number: 40


In [ ]:
print(days_log)

In [ ]:
class SingleTraitEmulator:
    def __init__(self, X, Y, kernel, noise=1e-10):
        self.model = GPyModelWrapper(GPRegression(X, Y, kernel, noise))
    
    def predict(self, X) -> Tuple[np.ndarray, np.ndarray]:
        return self.model.predict(X)
    
    def set_data(self, X: np.ndarray, Y: np.ndarray) -> None:
        self.model.set_data(X, Y)

#TODO: Data Preprocessing
X_size, Y_size = np.array([0,1,2,3]), np.array([1,3,7,2])
# Assuming a general quadratic relationship between mutation rate and survival years
size_kernel = GPy.kern.RatQuad(input_dim=1, power=1e4) * GPy.kern.Linear(1) * GPy.kern.Linear(-1)
size_emulator = SingleTraitEmulator(X_size, Y_size, size_kernel)

X_speed, Y_speed = np.array([0,1,2,3]), np.array([1,3,7,2])
speed_kernel = GPy.kern.RatQuad(input_dim=1, power=1e4) * GPy.kern.Linear(1) * GPy.kern.Linear(-1)
speed_emulator = SingleTraitEmulator(X_speed, Y_speed, speed_kernel)

X_vision, Y_vision = np.array([0,1,2,3]), np.array([1,3,7,2])
vision_kernel = GPy.kern.RatQuad(input_dim=1, power=1e4) * GPy.kern.Linear(1) * GPy.kern.Linear(-1)
vision_emulator = SingleTraitEmulator(X_vision, Y_vision, vision_kernel)

X_aggression, Y_agression = np.array([0,1,2,3]), np.array([1,3,7,2])
aggression_kernel = GPy.kern.RatQuad(input_dim=1, power=1e4) * GPy.kern.Linear(1) * GPy.kern.Linear(-1)
aggression_emulator = SingleTraitEmulator(X_aggression, Y_agression, aggression_kernel)

In [ ]:
# DGP using deepgp library

num_layers = 1
kern1 = GPy.kern.RBF(Q,ARD=True) + GPy.kern.Bias(num_layers)
kern2 = GPy.kern.RBF(X_tr.shape[1],ARD=False) + GPy.kern.Bias(X_tr.shape[1])
num_inducing = 40 # Number of inducing points to use for sparsification
back_constraint = False # Whether to use back-constraint for variational posterior
# encoder_dims=[[300],[150]] # Dimensions of the MLP back-constraint if set to true

mf_model = deepgp.DeepGP([Y_combined.shape[1], num_layeres, X_combined.shape[1]], Y_combined, X_combined, kernels=[kern1, kern2], num_inducing=num_inducing, back_constraint=back_constraint)

for i in range(len(mf_model.layers)):
    output_var = m.layers[i].Y.var() if i==0 else m.layers[i].Y.mean.var()
    m.layers[i].Gaussian_noise.variance = output_var*0.01
    m.layers[i].Gaussian_noise.variance.fix()

m.optimize(max_iters=800, messages=True)
for i in range(len(m.layers)):
    m.layers[i].Gaussian_noise.variance.unfix()

m.optimize(max_iters=1500, messages=True)

In [ ]:
# DGP using emukit and gpflow

def make_dgpMF_model(X=X_combined, Y=Y_combined):

    Din = X[0].shape[1]
    Dout = Y[0].shape[1]

    kernels = [RBF(Din, active_dims=list(range(Din)), variance=1., lengthscales=10., ARD=True)]
    for l in range(1,len(X)):
        D = Din + Dout
        D_range = list(range(D))
        
        k_corr_2 = RBF(Din, active_dims=D_range[:Din], lengthscales=0.1,  variance=1.5, ARD=True)
        k_corr = k_corr_2
        
        k_prev = RBF(Dout, active_dims=D_range[Din:], variance = 1., lengthscales=0.1, ARD=True)
        k_in = RBF(Din, active_dims=D_range[:Din], variance=0.1, lengthscales=1., ARD=True)
        
        k_bias = Linear(Dout, active_dims=D_range[Din:], variance = 1e-6)
        k_in.variance = 1e-6
        
        k_l = k_corr*(k_prev + k_bias) + k_in
        kernels.append(k_l)

    for i, kernel in enumerate(kernels[:-1]):
        kernels[i] += White(1, variance=0.)
            
    num_data = 0
    for i in range(len(X)):
        num_data += X[i].shape[0]
        
    layers = init_layers_mf(Y, X, kernels, num_outputs=1)
        
    model = DGP_Base(X, Y, Gaussian(), layers, num_samples=10, minibatch_size=1000)

    return model

def run(model, lr, iterations, callback=None):
    adam = AdamOptimizer(lr).make_optimize_action(model)
    actions = [adam] if callback is None else [adam, callback]
    loop = Loop(actions, stop=iterations)()
    model.anchor(model.enquire_session())
    
make_dgpMF_model([X_speed, X_combined], [Y_speed, Y_combined], [X_speed, X_combined])
dgp_model.layers[0].feature.Z.trainable = False
dgp_model.layers[1].feature.Z.trainable = False
dgp_model.layers[0].q_sqrt.trainable = False
dgp_model.likelihood.likelihood.variance.trainable = False
dgp_model.run(0.01, 1500)

In [ ]:
# BayesOpt for low fidelity emulators

_, space = forrester_function()
iterations = 10

ei_acquisition = ExpectedImprovement(speed_model)
pi_acquisition = ProbabilityOfImprovement(speed_model)
ucb_acquisition = NegativeLowerConfidenceBound(speed_model)

for _ in range(iterations):
    optimizer = GradientAcquisitionOptimizer(space)
    x_new, _ = optimizer.optimize(ei_acquisition)
    mutation_rate = {"speed":x_new, "size":0, "vision":0, "aggression":0,}
    y_new = Simulator(mutation_rate)
    X = np.append(X, x_new)
    Y = np.append(Y, y_new)
    speed_model.set_data(X, Y)

In [ ]:
# Sensitivity Analysis
senstivity = MonteCarloSensitivity(model = speed_model, input_domain = space)
main_effects, total_effects, _ = senstivity.compute_effects(num_monte_carlo_points = 10000)